In [1]:
import sys
import os


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from app.database import AsyncSessionLocal


In [2]:
from langchain_core.tools import tool

from langgraph.prebuilt import InjectedState, ToolNode

from langchain_core.prompts import ChatPromptTemplate # Ou PromptTemplate
from langchain_openai import ChatOpenAI

from app.api.schemas.company_profile import CompanyProfileSchema

In [12]:
from typing import List, Optional
from pydantic import BaseModel, Field, HttpUrl # Supondo que OfferingInfo e CompanyProfileSchema estão definidos como você mostrou

def generate_system_message(profile: CompanyProfileSchema) -> str:
    # --- Seção 1: Identidade e Persona do AI ---
    persona_intro = f"Você é um assistente de vendas virtual especialista da empresa '{profile.company_name}'."
    if profile.target_audience:
        persona_intro += f" Seu foco é atender {profile.target_audience}."
    
    tone_instruction = f"Seu tom de comunicação deve ser {profile.sales_tone}."
    language_instruction = f"Comunique-se primariamente em {profile.language}."

    # --- Seção 2: Sobre a Empresa ---
    company_description = f"A '{profile.company_name}' {profile.business_description}."
    if profile.website:
        company_description += f" Você pode encontrar mais informações em nosso site: {profile.website}."
    if profile.address: # Apenas inclua se for relevante para o tipo de negócio
        company_description += f" Nosso endereço físico é: {profile.address}."
    if profile.opening_hours:
        company_description += f" Nosso horário de funcionamento é: {profile.opening_hours}."


    # --- Seção 3: Objetivos e Estratégia de Vendas do AI ---
    objective_statement = f"Seu principal objetivo como AI é: {profile.ai_objective}."

    key_selling_points_list = ""
    if profile.key_selling_points:
        points_str = "\n".join([f"- {point}" for point in profile.key_selling_points])
        key_selling_points_list = f"Nossos principais diferenciais e pontos de venda são:\n{points_str}"

    # --- Seção 4: Visão Geral das Ofertas (Produtos/Serviços) ---
    # Esta seção pode ficar longa. Considere se é melhor que o AI use uma TOOL para buscar detalhes,
    # mas uma visão geral aqui é útil.
    offerings_summary_parts = ["Aqui está um resumo de nossas principais ofertas:"]
    if profile.offering_overview:
        for offer in profile.offering_overview:
            offer_detail = f"\n\n**Oferta: {offer.name}**"
            offer_detail += f"\n  Oferta ID: {offer.id}**"
            offer_detail += f"\n  Descrição: {offer.short_description}"
            if offer.price:
                offer_detail += f"\n  Preço: {offer.price}"
            offerings_summary_parts.append(offer_detail)
        offerings_summary_parts.append("\nPara detalhes completos sobre qualquer oferta específica (incluindo todas as características, preços atuais, bônus ou links diretos), você DEVE usar a ferramenta 'get_offering_details'.")
    else:
        offerings_summary_parts.append("Informações sobre ofertas específicas devem ser recuperadas usando a ferramenta 'get_offering_details' quando um cliente perguntar.")

    
    offerings_summary = "".join(offerings_summary_parts)
    
    delivery_info = ""
    if profile.delivery_options:
        delivery_str = ", ".join(profile.delivery_options)
        delivery_info = f"Oferecemos as seguintes opções de entrega/retirada: {delivery_str}."

    # --- Seção 5: Diretrizes de Comunicação e Comportamento ---
    # Adicionando princípios gerais de vendas que discutimos
    general_sales_principles = [
        "Adote os princípios SNAP: mantenha suas respostas Simples e diretas, seja iNestimável fornecendo valor rapidamente, sempre se Alinhe com as necessidades do cliente e ajude a elevar as Prioridades dele.",
        "Quando apropriado, faça perguntas abertas e concisas (inspirado no SPIN Selling leve) para entender a Situação do cliente, os Problemas que enfrenta, as Implicações desses problemas e a Necessidade de uma solução.",
        "Seja proativo em sugerir o próximo passo lógico e em fornecer informações que o cliente pode não ter pensado em pedir.",
        "Mantenha o controle da conversa de forma sutil, sempre buscando adicionar valor e mover a interação adiante.",
        "Após responder a uma pergunta, sempre considere como você pode adicionar valor e guiar a conversa. Você pode fazer uma pergunta de follow-up relevante, sugerir um próximo passo lógico, ou conectar a resposta a um benefício chave."
    ]
    
    communication_rules_list = ["\nDiretrizes de Comunicação Específicas:"]
    if profile.communication_guidelines:
        for guideline in profile.communication_guidelines:
            communication_rules_list.append(f"- {guideline}")
    else:
        communication_rules_list.append("- Siga as melhores práticas gerais de atendimento ao cliente.")
    
    # Combinando princípios gerais com diretrizes específicas
    all_guidelines = "\n".join(general_sales_principles) + "\n" + "\n".join(communication_rules_list)


    # --- Seção 6: Uso de Ferramentas (Genérico, pois as tools são bindadas separadamente) ---
    tools_mention = (
        "\nVocê tem acesso a ferramentas para buscar informações detalhadas de produtos/serviços, "
        "responder a perguntas, gerar links de compra, "
        "pensar no próximo passo estrategicamente. Use-as sempre que necessário para fornecer "
        "respostas precisas e eficientes e para executar ações solicitadas."
    )

    # --- Seção 7: Fallback e Escalonamento ---
    fallback_instruction = ""
    if profile.fallback_contact_info:
        fallback_instruction = (
            f"\nSe você não puder ajudar diretamente ou se o cliente solicitar, "
            f"você pode fornecer as seguintes informações de contato ou direcionamento: '{profile.fallback_contact_info}'. "
            # "Se o cliente solicitar falar com um humano, facilite essa transição ou use a ferramenta de agendamento de handoff."
        )
    else:
        fallback_instruction = (
            "\nSe você não puder ajudar diretamente, peça desculpas e informe que você não tem a informação no momento. "
            # "Se o cliente solicitar falar com um humano, use a ferramenta de agendamento de handoff."
        )

    # --- Montagem Final do System Message ---
    system_message_parts = [
        persona_intro,
        tone_instruction,
        language_instruction,
        "\n--- Sobre Nós ---",
        company_description,
        "\n--- Seu Papel e Objetivos ---",
        objective_statement,
        key_selling_points_list if profile.key_selling_points else "",
        "\n--- Nossas Ofertas e Serviços ---",
        offerings_summary,
        delivery_info if profile.delivery_options else "",
        "\n--- Como Você Deve Se Comunicar e Agir ---",
        all_guidelines,
        tools_mention,
        fallback_instruction,
        "\nLembre-se, seu objetivo é ser um consultor de vendas eficaz, ajudando os clientes e representando bem a marca.",
    ]
    
    # Remove partes vazias e junta tudo
    final_system_message = "\n".join(filter(None, system_message_parts))
    
    return final_system_message

# --- Exemplo de Uso (requer que CompanyProfileSchema e OfferingInfo estejam definidos) ---
# Supondo que você tenha uma instância 'company_profile_data' do tipo CompanyProfileSchema
# Exemplo:
# company_profile_data = CompanyProfileSchema(**CompanyProfileSchema.model_config["json_schema_extra"]["example"])
#
# system_prompt_string = generate_system_message(company_profile_data)
# print(system_prompt_string)
#
# from langchain_core.messages import SystemMessage
# sys_msg_object = SystemMessage(content=system_prompt_string)

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:

from sqlalchemy.ext.asyncio import async_sessionmaker, AsyncSession

from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
from langgraph.checkpoint.serde.jsonplus import JsonPlusSerializer
from langchain_openai import AzureChatOpenAI
from langchain_core.language_models import BaseChatModel

from app.database import AsyncSessionLocal
from app.config import get_settings  # Use the centralized settings
settings = get_settings()

llm_primary_client = AzureChatOpenAI(
    azure_deployment=settings.PRIMARY_LLM_MODEL_NAME,
    temperature=settings.PRIMARY_LLM_TEMPERATURE,
    azure_endpoint=settings.AZURE_OPENAI_ENDPOINT,
    api_key=settings.AZURE_OPENAI_API_KEY,
    api_version="2025-01-01-preview",
)

db_session_factory: Optional[async_sessionmaker[AsyncSession]] = AsyncSessionLocal
llm_primary_client: Optional[BaseChatModel] = llm_primary_client
llm_fast_client: Optional[BaseChatModel] = llm_primary_client



In [6]:
from app.services.helper.checkpoint import reset_checkpoint
async with AsyncSessionLocal() as db:
    await reset_checkpoint(db=db, thread_id="84d6ef59-c6ac-4d13-ac66-016e344f1654")
    await db.commit()

2025-06-01 03:23:10.588 | INFO     | app.services.helper.checkpoint:reset_checkpoint:22 - [reset_checkpoint] Deleting checkpoint data for thread_id: 84d6ef59-c6ac-4d13-ac66-016e344f1654...
2025-06-01 03:23:10.588 | DEBUG    | app.services.helper.checkpoint:reset_checkpoint:26 - [reset_checkpoint] Deleting from checkpoint_writes...


2025-06-01 03:23:10,740 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-01 03:23:10,742 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-01 03:23:10,788 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-01 03:23:10,791 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-01 03:23:10,793 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-01 03:23:10,793 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-01 03:23:10,794 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-01 03:23:10,795 INFO sqlalchemy.engine.Engine DELETE FROM checkpoint_writes WHERE thread_id = $1
2025-06-01 03:23:10,795 INFO sqlalchemy.engine.Engine [generated in 0.00035s] ('84d6ef59-c6ac-4d13-ac66-016e344f1654',)


2025-06-01 03:23:10.798 | DEBUG    | app.services.helper.checkpoint:reset_checkpoint:29 - [reset_checkpoint] Deleted rows from checkpoint_writes (if any existed).
2025-06-01 03:23:10.798 | DEBUG    | app.services.helper.checkpoint:reset_checkpoint:26 - [reset_checkpoint] Deleting from checkpoint_blobs...


2025-06-01 03:23:10,799 INFO sqlalchemy.engine.Engine DELETE FROM checkpoint_blobs WHERE thread_id = $1
2025-06-01 03:23:10,799 INFO sqlalchemy.engine.Engine [generated in 0.00041s] ('84d6ef59-c6ac-4d13-ac66-016e344f1654',)


2025-06-01 03:23:10.801 | DEBUG    | app.services.helper.checkpoint:reset_checkpoint:29 - [reset_checkpoint] Deleted rows from checkpoint_blobs (if any existed).
2025-06-01 03:23:10.801 | DEBUG    | app.services.helper.checkpoint:reset_checkpoint:26 - [reset_checkpoint] Deleting from checkpoints...


2025-06-01 03:23:10,802 INFO sqlalchemy.engine.Engine DELETE FROM checkpoints WHERE thread_id = $1
2025-06-01 03:23:10,802 INFO sqlalchemy.engine.Engine [generated in 0.00063s] ('84d6ef59-c6ac-4d13-ac66-016e344f1654',)


2025-06-01 03:23:10.807 | DEBUG    | app.services.helper.checkpoint:reset_checkpoint:29 - [reset_checkpoint] Deleted rows from checkpoints (if any existed).


2025-06-01 03:23:10,809 INFO sqlalchemy.engine.Engine COMMIT


In [10]:


from langgraph.prebuilt import create_react_agent
from app.services.repository import conversation as conversation_repo
from app.services.repository import bot_agent as bot_agent_repo
from app.services.repository import company_profile as profile_repo

from app.services.sales_agent.tools.knowledge import query_knowledge_base
from app.services.sales_agent.tools.offering import get_offering_details_by_id, update_shopping_cart,generate_checkout_link_for_cart
from app.services.sales_agent.tools.strategy import suggest_objection_response_strategy

from app.api.schemas.bot_agent import BotAgentRead
from app.services.sales_agent.agent_state import  AgentState
from uuid import  UUID
from loguru import logger

from langchain_core.messages import HumanMessage
from langgraph.managed.is_last_step import RemainingSteps
from typing import Any, Tuple

from pydantic import HttpUrl, EmailStr
class JsonOnlySerializer(JsonPlusSerializer):
    def _default(self, o: Any) -> Any:
        # Trate primeiro os tipos mais específicos
        if isinstance(o, HttpUrl):
            return str(o)
        if isinstance(o, EmailStr):
            return str(o)
            
        
        # Se não for HttpUrl nem EmailStr, delegue para a implementação-pai
        try:
            return super()._default(o)
        except AttributeError:
            # Se a classe-pai não tiver _default, lance um erro indicando que
            # não sabe como serializar o objeto em JSON
            raise TypeError(f"Objeto do tipo {o.__class__.__name__} não é serializável pelo JsonOnlySerializer")
    
    def dumps_typed(self, obj: Any) -> Tuple[str, bytes]:
        # Retorna sempre JSON
        return "json", self.dumps(obj)

serializer = JsonOnlySerializer()

# Ensure DATABASE_URL is correctly formatted for AsyncPostgresSaver
db_conn_string_pg = str(settings.DATABASE_URL).replace(
    "postgresql+asyncpg://", "postgresql://"
)


account_id = UUID("12996a09-b660-4156-95d0-b56e7d8dc3d4")
conversation_id = UUID("84d6ef59-c6ac-4d13-ac66-016e344f1654")
async with AsyncSessionLocal() as db:
    company_profile = await profile_repo.get_profile_by_account_id(db=db, account_id=account_id)

profile = CompanyProfileSchema.model_validate(company_profile)   
prompt = generate_system_message(profile=profile)



async with AsyncPostgresSaver.from_conn_string(
    db_conn_string_pg, serde=serializer
) as checkpointer:

    tools=[query_knowledge_base, get_offering_details_by_id, update_shopping_cart,generate_checkout_link_for_cart, suggest_objection_response_strategy]

    compiled_reply_graph = create_react_agent(
        model=llm_primary_client.bind_tools(tools, parallel_tool_calls=False),
        tools=tools,
        state_schema=AgentState,
        prompt=prompt,
        checkpointer=checkpointer
    )


    async with db_session_factory() as db:
        
        conversation = await conversation_repo.find_conversation_by_id(
            db, account_id=account_id, conversation_id=conversation_id
        )
        profile_db = await profile_repo.get_profile_by_account_id(
            db, account_id=account_id
        )

        agent_config_db = await bot_agent_repo.get_bot_agent_for_inbox(
                db, inbox_id=conversation.inbox_id, account_id=account_id
            )
        profile_dict = CompanyProfileSchema.model_validate(
            profile_db
        ).model_dump(mode="json")

        agent_config_dict = BotAgentRead.model_validate(agent_config_db).model_dump(mode="json")

        current_input: AgentState = {
            "account_id": account_id,
            "conversation_id": conversation_id,
            "bot_agent_id": agent_config_db.id,
            "company_profile": profile_dict,
            "agent_config": agent_config_dict,
            "trigger_event": "user_message",
            "remaining_steps": RemainingSteps(),

        }
        AgentState.model_validate(current_input)

        config = {
            "configurable": {
                "thread_id": str(conversation_id),
                "llm_primary_instance": llm_primary_client,
                "llm_fast_instance": llm_fast_client,
                "db_session_factory": db_session_factory,  # Pass the factory
            }
        }

        # messages = [HumanMessage(content="como funciona a politica de reembolso?")]
        logger.info(f"current input : {current_input}")
        
        current_input["messages"] = [HumanMessage(content="mas é mto caro! ")]
        # print(current_input)
        # AgentState.model_validate(current_input)
        messages = await compiled_reply_graph.ainvoke(current_input, config)
    

# await checkpointer.connection.close() 
   
    
        



2025-06-01 03:26:25.384 | DEBUG    | app.services.repository.company_profile:get_profile_by_account_id:29 - Fetching company profile for account_id: 12996a09-b660-4156-95d0-b56e7d8dc3d4


2025-06-01 03:26:25,393 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-01 03:26:25,395 INFO sqlalchemy.engine.Engine SELECT company_profiles.id, company_profiles.account_id, company_profiles.company_name, company_profiles.website, company_profiles.address, company_profiles.business_description, company_profiles.target_audience, company_profiles.sales_tone, company_profiles.language, company_profiles.communication_guidelines, company_profiles.ai_objective, company_profiles.key_selling_points, company_profiles.offering_overview, company_profiles.delivery_options, company_profiles.opening_hours, company_profiles.fallback_contact_info, company_profiles.profile_version, company_profiles.created_at, company_profiles.updated_at 
FROM company_profiles 
WHERE company_profiles.account_id = $1::UUID
2025-06-01 03:26:25,395 INFO sqlalchemy.engine.Engine [cached since 60.91s ago] (UUID('12996a09-b660-4156-95d0-b56e7d8dc3d4'),)


2025-06-01 03:26:25.399 | DEBUG    | app.services.repository.company_profile:get_profile_by_account_id:36 - Found profile with id: 4cb673cf-6d21-431f-b212-f080e1d01a38


2025-06-01 03:26:25,399 INFO sqlalchemy.engine.Engine ROLLBACK
2025-06-01 03:26:25,422 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-01 03:26:25,423 INFO sqlalchemy.engine.Engine SELECT conversations.id, conversations.account_id, conversations.inbox_id, conversations.contact_inbox_id, conversations.assignee_id, conversations.display_id, conversations.user_last_seen_at, conversations.agent_last_seen_at, conversations.status, conversations.unread_agent_count, conversations.is_bot_active, conversations.is_simulation, conversations.locked, conversations.last_message_at, conversations.additional_attributes, conversations.created_at, conversations.updated_at 
FROM conversations 
WHERE conversations.id = $1::UUID AND conversations.account_id = $2::UUID
2025-06-01 03:26:25,425 INFO sqlalchemy.engine.Engine [cached since 60.91s ago] (UUID('84d6ef59-c6ac-4d13-ac66-016e344f1654'), UUID('12996a09-b660-4156-95d0-b56e7d8dc3d4'))
2025-06-01 03:26:25,427 INFO sqlalchemy.engine.Engine SELECT i

2025-06-01 03:26:25.433 | DEBUG    | app.services.repository.conversation:find_conversation_by_id:49 - [conversation] Found conversation 84d6ef59-c6ac-4d13-ac66-016e344f1654
2025-06-01 03:26:25.434 | DEBUG    | app.services.repository.company_profile:get_profile_by_account_id:29 - Fetching company profile for account_id: 12996a09-b660-4156-95d0-b56e7d8dc3d4


2025-06-01 03:26:25,434 INFO sqlalchemy.engine.Engine SELECT company_profiles.id, company_profiles.account_id, company_profiles.company_name, company_profiles.website, company_profiles.address, company_profiles.business_description, company_profiles.target_audience, company_profiles.sales_tone, company_profiles.language, company_profiles.communication_guidelines, company_profiles.ai_objective, company_profiles.key_selling_points, company_profiles.offering_overview, company_profiles.delivery_options, company_profiles.opening_hours, company_profiles.fallback_contact_info, company_profiles.profile_version, company_profiles.created_at, company_profiles.updated_at 
FROM company_profiles 
WHERE company_profiles.account_id = $1::UUID
2025-06-01 03:26:25,434 INFO sqlalchemy.engine.Engine [cached since 60.95s ago] (UUID('12996a09-b660-4156-95d0-b56e7d8dc3d4'),)


2025-06-01 03:26:25.435 | DEBUG    | app.services.repository.company_profile:get_profile_by_account_id:36 - Found profile with id: 4cb673cf-6d21-431f-b212-f080e1d01a38
2025-06-01 03:26:25.435 | DEBUG    | app.services.repository.bot_agent:get_bot_agent_for_inbox:264 - Checking for assigned BotAgent for Inbox 186dbc62-39b3-4eab-851c-d95a8db769f9 in Account 12996a09-b660-4156-95d0-b56e7d8dc3d4


2025-06-01 03:26:25,436 INFO sqlalchemy.engine.Engine SELECT bot_agents.id, bot_agents.account_id, bot_agents.name, bot_agents.first_message, bot_agents.use_rag, bot_agents.created_at, bot_agents.updated_at 
FROM bot_agents JOIN bot_agent_inboxes ON bot_agents.id = bot_agent_inboxes.bot_agent_id 
WHERE bot_agent_inboxes.inbox_id = $1::UUID AND bot_agent_inboxes.account_id = $2::UUID
2025-06-01 03:26:25,436 INFO sqlalchemy.engine.Engine [cached since 60.89s ago] (UUID('186dbc62-39b3-4eab-851c-d95a8db769f9'), UUID('12996a09-b660-4156-95d0-b56e7d8dc3d4'))
2025-06-01 03:26:25,437 INFO sqlalchemy.engine.Engine SELECT accounts.id AS accounts_id, accounts.stripe_customer_id AS accounts_stripe_customer_id, accounts.name AS accounts_name, accounts.locale AS accounts_locale, accounts.active_plan_tier AS accounts_active_plan_tier, accounts.simulation_inbox_id AS accounts_simulation_inbox_id, accounts.simulation_contact_id AS accounts_simulation_contact_id, accounts.simulation_conversation_id AS a

2025-06-01 03:26:25.438 | DEBUG    | app.services.repository.bot_agent:get_bot_agent_for_inbox:277 - Inbox 186dbc62-39b3-4eab-851c-d95a8db769f9 is assigned to BotAgent b39a001b-f4d4-4d26-8194-fe35ee23c8e3
2025-06-01 03:26:25.438 | INFO     | __main__:<module>:114 - current input : {'account_id': UUID('12996a09-b660-4156-95d0-b56e7d8dc3d4'), 'conversation_id': UUID('84d6ef59-c6ac-4d13-ac66-016e344f1654'), 'bot_agent_id': UUID('b39a001b-f4d4-4d26-8194-fe35ee23c8e3'), 'company_profile': {'company_name': 'APP A Minha Casa', 'website': None, 'address': '', 'business_description': 'Aplicativo pessoal do Ícaro de Carvalho focado em questões pessoais: família, casamento, criação de filhos, fé, dinheiro, propósito de vida e busca pela felicidade. Oferece conteúdos para desenvolvimento pessoal e equilíbrio em diversas áreas da vida.', 'target_audience': 'Pessoas interessadas em desenvolvimento pessoal, família, casamento, finanças, fé e propósito de vida.', 'sales_tone': 'amigável, prestativo, p

2025-06-01 03:26:39,826 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
for m in messages['messages']:
    m.pretty_print()
    

================================ Human Message =================================

ola queria um produto do icaro, mas é mto caro. 
================================== Ai Message ==================================

Entendo que o preço pode ser uma preocupação. O Pacote de Desenvolvimento Pessoal do Ícaro de Carvalho é uma oferta abrangente que inclui cursos, masterclasses e conteúdos exclusivos para ajudar no desenvolvimento pessoal e equilíbrio em diversas áreas da vida. 

Para entender melhor suas necessidades, você poderia me contar um pouco mais sobre o que está buscando? Assim, posso ajudar a ver como essa oferta pode agregar valor para você. Além disso, posso verificar se há alguma promoção ou benefício adicional que possa tornar a compra mais acessível.
================================ Human Message =================================

sim queria cuidar de minha fincancas e minha casa 
================================== Ai Message ==================================

Cuidar das finan

In [21]:
 messages['messages']

[HumanMessage(content='consegue gerar os links de todos meus items do carrinho?', additional_kwargs={}, response_metadata={}, id='100d494e-5497-433c-81a4-1df0392f3945'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AA8FqcJmxiIFzFth0w7TriiS', 'function': {'arguments': '{}', 'name': 'generate_checkout_link_for_cart'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 1621, 'total_tokens': 1635, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1536}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BdS6B2Ds3BURaVHSdQLlvKKSfV9VH', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexu